<a href="https://colab.research.google.com/github/ekaterinaKrupkina/decomposer_notebooks/blob/main/Run_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
import os
import subprocess
import json
from google.colab import userdata
import shlex
import pandas as pd
import time
import sys


## Set up environment

In [2]:
!rm -rf /usr/local/*

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%%time
if not os.path.exists("/usr/local/venv-deployed"):
  !tar -xf /content/drive/MyDrive/HarmBench/venv.tar.gz -C /usr/local
  !touch /usr/local/venv-deployed

CPU times: user 470 ms, sys: 84.1 ms, total: 554 ms
Wall time: 1min 30s


In [ ]:
# !tar -xf /content/drive/MyDrive/venv.tar.gz -C /usr/local

In [4]:
sys.path.append('/usr/local/lib/python3.10/site-packages')

In [ ]:
sys.path

['/content',
 '/env/python',
 '/usr/lib/python310.zip',
 '/usr/lib/python3.10',
 '/usr/lib/python3.10/lib-dynload',
 '',
 '/usr/local/lib/python3.10/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.10/dist-packages/IPython/extensions',
 '/root/.ipython']

In [ ]:
## test code
#import weave

In [ ]:
!chmod +x /usr/local/bin/pip


In [ ]:
os.environ["WANDB__EXECUTABLE"] = "/usr/local/bin/python"

In [ ]:
os.environ["WANDB_API_KEY"] = userdata.get("WANDB_API_KEY")
os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")
os.environ["GITHUB_TOKEN"] = userdata.get("GITHUB_TOKEN")
os.environ["OPENAI_BASE_URL"] = "https://api.openai.com/v1"
os.environ["MISTRAL_API_KEY"] = userdata.get("MISTRAL_API_KEY")

In [ ]:
if not os.path.exists("HarmBench"):
  !git clone -q --recursive git@github.com:ekaterinaKrupkina/HarmBench
%cd /content/HarmBench

In [ ]:
if not os.path.exists("Decomposition-Harness"):
  !git clone https://{userdata.get("GITHUB_TOKEN")}@github.com/PalisadeResearch/Decomposition-Harness

Cloning into 'Decomposition-Harness'...
remote: Enumerating objects: 312, done.
remote: Counting objects: 100% (94/94), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 312 (delta 57), reused 51 (delta 31), pack-reused 218
Receiving objects: 100% (312/312), 923.51 KiB | 3.52 MiB/s, done.
Resolving deltas: 100% (149/149), done.


In [ ]:
import vllm

In [ ]:
%cd /content/drive/MyDrive/HarmBench

/content/drive/MyDrive/HarmBench


## Some important funcs

In [ ]:
def run_in_background(script_path, output_file='output.log'):
    """
    Run a Python script in the background and redirect its output to a file.

    This function executes the specified Python script as a separate process,
    allowing it to run independently of the calling process. The script's
    standard output and standard error are redirected to the specified output file.

    Parameters:
    script_path (str): The path to the Python script to be executed.
    output_file (str, optional): The path to the file where the script's output
                                 will be written. Defaults to 'output.log'.

    Returns:
    int: The process ID (PID) of the newly created background process.

    Notes:
    - The function uses subprocess.Popen to create a new process.
    - shlex.split is used to properly parse the command arguments.
    - The script's stderr is redirected to stdout, which is then written to the output file.
    - The process is set to run independently of the parent process using os.setsid.
    - The function prints the PID of the new process to the console.

    Raises:
    subprocess.SubprocessError: If there's an error in creating or running the subprocess.
    IOError: If there's an error in opening or writing to the output file.

    Example:
    >>> pid = run_in_background('/path/to/script.py', 'script_output.log')
    >>> print(f"Script is running in background with PID: {pid}")
    """
    # Construct the command
    command = f"python {script_path}"

    with open(output_file, 'a') as f:
        process = subprocess.Popen(
            shlex.split(command),  # Use shlex.split for proper argument parsing
            stdout=f,
            stderr=subprocess.STDOUT,  # Redirect stderr to stdout
            preexec_fn=os.setsid  # This allows the process to run independently of the parent
        )

    print(f"Process started with PID: {process.pid}")
    return process.pid

In [ ]:
def run_generate_completions(model_name, models_config_file, test_cases_path, save_path, behaviors_path):
    """
    Run the generate_completions.py script as a subprocess.

    This function executes the generate_completions.py script with the specified parameters,
    captures its output in real-time, and prints it to the console.

    Args:
        model_name (str): Name of the model to use for generating completions.
        models_config_file (str): Path to the models configuration file.
        test_cases_path (str): Path to the test cases file.
        save_path (str): Path where the generated completions will be saved.
        behaviors_path (str): Path to the behaviors dataset file.

    Returns:
        None

    Raises:
        subprocess.CalledProcessError: If the subprocess returns a non-zero exit status.
    """
    command = [
        "python", "-u", "generate_completions.py",
        f"--model_name={model_name}",
        f"--models_config_file={models_config_file}",
        f"--test_cases_path={test_cases_path}",
        f"--save_path={save_path}",
        f"--behaviors_path={behaviors_path}"
    ]

    process = subprocess.Popen(
        command,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        text=True
    )

    for line in process.stdout:
        print(line, end='')

    return_code = process.wait()

    if return_code != 0:
        print(f"Process exited with return code {return_code}")

In [ ]:
def run_evaluate_completions(behaviors_path, completions_path, save_path):
    """
    Run the evaluate_completions.py script as a subprocess.

    This function executes the evaluate_completions.py script with the specified parameters,
    captures its output in real-time, and prints it to the console.

    Args:
        behaviors_path (str): Path to the behaviors dataset file.
        completions_path (str): Path to the completions file to be evaluated.
        save_path (str): Path where the evaluation results will be saved.

    Returns:
        None

    Raises:
        subprocess.CalledProcessError: If the subprocess returns a non-zero exit status.
    """
    command = [
        sys.executable,
        "-u",
        "evaluate_completions.py",
        f"--behaviors_path={behaviors_path}",
        f"--completions_path={completions_path}",
        f"--save_path={save_path}"
    ]

    try:
        process = subprocess.Popen(
            command,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT,
            text=True
        )

        for line in process.stdout:
            print(line, end='')

    except subprocess.CalledProcessError as e:
        print(f"Error: {e}")

In [ ]:
def experiment(experiment_params):
    """
    Run a complete experiment pipeline including completion generation, evaluation, and result processing.

    This function sets up the environment, runs the necessary scripts for generating completions
    and evaluating them, and then processes and logs the results.

    Args:
        experiment_params (dict): A dictionary containing experiment parameters including model names.

    Returns:
        None
    """

    for k, v in experiment_params.items():
        os.environ[k] = v

    os.environ["OPENAI_BASE_URL"] = "https://api.openai.com/v1"
    run_in_background("./Decomposition-Harness/server/web.py", "output.log")

    # TODO for Vladimir? create an empty file after initialization
    #
    time.sleep(10) # waiting for decomposer server initialization

    os.environ["OPENAI_BASE_URL"] = "http://127.0.0.1:8082"

    model_name = os.getenv("TARGET_MODEL")
    try:
      run_generate_completions(
          model_name=model_name,
          models_config_file="./configs/model_configs/models.yaml",
          test_cases_path="./results/test_cases/test_cases_shortest.json", # change to full
          save_path=f"./results/completions/completions_{model_name}.json",
          behaviors_path="./data/behavior_datasets/harmbench_behaviors_text_all.csv"
      )

      run_evaluate_completions(
          behaviors_path="./data/behavior_datasets/harmbench_behaviors_text_all.csv",
          completions_path=f"./results/completions/completions_{model_name}.json",
          save_path=f"./default/results/evaled_{model_name}.json"
      )

      process_results(f"./default/results/evaled_{model_name}", decomposer=True)

    except Exception as e:
      print(e)
      pass



    !pkill --full 'web\.py'


In [ ]:
def experiment(experiment_params):
    """
    Run a complete experiment pipeline including completion generation, evaluation, and result processing.

    This function sets up the environment, runs the necessary scripts for generating completions
    and evaluating them, and then processes and logs the results.

    Args:
        experiment_params (dict): A dictionary containing experiment parameters including model names.

    Returns:
        None
    """

    for k, v in experiment_params.items():
        os.environ[k] = v

    os.environ["OPENAI_BASE_URL"] = "https://api.openai.com/v1"
    run_in_background("./Decomposition-Harness/server/web.py", "output.log")
    time.sleep(10)
    os.environ["OPENAI_BASE_URL"] = "http://127.0.0.1:8082"

    model_name = os.getenv("TARGET_MODEL")

    run_generate_completions(
        model_name=model_name,
        models_config_file="./configs/model_configs/models.yaml",
        test_cases_path="./results/test_cases/test_cases_short.json",
        save_path=f"./results/completions/completions_{model_name}.json",
        behaviors_path="./data/behavior_datasets/harmbench_behaviors_text_all.csv"
    )

    run_evaluate_completions(
        behaviors_path="./data/behavior_datasets/harmbench_behaviors_text_all.csv",
        completions_path=f"./results/completions/completions_{model_name}.json",
        save_path=f"./default/results/evaled_{model_name}.json"
    )

    #process_results(f"evaled_{model_name}", decomposer=True)

    !pkill --full 'web\.py'


In [ ]:
!sed -i 's#API_TIMEOUT = 20#API_TIMEOUT = 600#' /content/drive/MyDrive/HarmBench/baselines/pair/language_models.py

In [ ]:
!pkill --full 'web\.py'

In [ ]:
%cd /content/drive/MyDrive/HarmBench

## Run experiments

In [ ]:
experiment_params = {
    "TARGET_MODEL": "gpt-4o-2024-05-13",
    "COMPOSER_MODEL": "mistral-small-2312",
    "DECOMPOSER_MODEL": "mistral-small-2312",
    "SURROGATE_MODEL": "mistral-small-2312"
    }

In [ ]:
experiment(experiment_params)

Process started with PID: 10037
Namespace(model_name='gpt-4o-2024-05-13', models_config_file='./configs/model_configs/models.yaml', behaviors_path='./data/behavior_datasets/harmbench_behaviors_text_all.csv', test_cases_path='./results/test_cases/test_cases_short.json', save_path='./results/completions/completions_gpt-4o-2024-05-13.json', max_new_tokens=256, generate_with_vllm=False, incremental_update=False)
model_config {'model_name_or_path': 'gpt-4o', 'token': '<your_openai_token>', 'num_gpus': 1}
Generating completions for 40 test cases
Generating completions...

  0%|          | 0/40 [00:00<?, ?it/s]<class 'openai.APIConnectionError'> Connection error.
<class 'openai.APIConnectionError'> Connection error.


KeyboardInterrupt: 



---

